In [ ]:
Name: Chang Liu
For the code challenges, some of them I did it with python, some with Go. 


In [ ]:
1.1.1 pattern count
# Insert your PatternCount function here, along with any subroutines you need
def pattern_count(text: str, pattern: str) -> int:
    count = 0
    for i in range(len(text)-len(pattern)+1):
        if text[i:i+len(pattern)] == pattern:
            count += 1
    return count


In [ ]:
1.1.2 profile-most probable k-mers
def profile_most_probable_kmer(text: str, k: int, profile: list[dict[str, float]]) -> str:
    #define a initial probability that cannot be worse
    bestprob = -1
    best =""
    # traverse every possible start point (n-k+1) of the k-mer
    num = len(text)-k+1
    for i in range(num):
        prob = 1.0
        motif = text[i:i+k]
 # calculate the product of every column probability
        for j in range(k):
            prob *= profile[j][motif[j]]
        if prob > bestprob:
            best = motif
            bestprob = prob
    return best



In [ ]:
1.1.3 Randomized Motif Finding 1000 times
import sys
import random

def randomized_motif_search(dna: list[str], k: int, t: int) -> list[str]:
    """Implements the RandomizedMotifSearch algorithm with pseudocounts."""
    # define the initial score as the worst case so it can updated in the following steps
    bestscore = k*t
    bestmotif = " "
    # run the randomized_motif_search function 1000 times, and keep the motifs that has the best score (Smallest)
    for i in range(1000):
        motifs = randomized_motif_once(dna, k, t)
        if score(motifs) < bestscore:
            bestmotif = motifs
            bestscore = score(motifs)
    return bestmotif

# runs a single randomized_motif_search
def randomized_motif_once(dna: list[str], k: int, t:int) -> list[str]:
    initial = randomizedInitial(dna, k, t)
    best = initial
    # generate a forever loop, only break when the score doesn’t improve.
    while True:
 # generate the profile matrix for the current motifs
        profile = Profile(best)
 # find out the most probable motifs of each seq based on the profile matrix
        motif = Motif(dna, k, profile)
 # update if score improves
        if score(motif) < score(best):
            best = motif
        else:
            break
        return motif


# generate a random k-mer for each seq of the dna list 
def randomizedInitial(dna: list[str], k: int, t:int) -> list[str]:
    initial = []
    # for each DNA, generate a random number as the start point of the random k-mer
    # the random number is within [0,n-k+1](inclusive) 
    for i in range(t):
        index = random.randint(0, len(dna[i])-k-1)
        text = dna[i]
        initial.append(text[index:index+k])
    return initial


# generate the probability of each nucleotide in each column
def Profile(motif: list[str]) -> list[dict[str, float]]:
    profile = []
    # for each column
    for j in range(len(motif[0])):
        dictionary = {"A":1,"C":1,"G":1,"T":1}
        ## considering pseudocount
        total = 4 + len(motif)
        # for each row, count the frequency of each nucleotide
        for i in range(len(motif)):
            text = motif[i]
            dictionary[motif[i][j]] += 1
        # calculate the probability
        dictionary["A"] = dictionary["A"]/total
        dictionary["C"] = dictionary["C"]/total
        dictionary["G"] = dictionary["G"]/total
        dictionary["T"] = dictionary["T"]/total
        profile.append(dictionary)
    return profile


# input the profile matrix of a set of motifs and a DNA seq, output the most-probable k-mer
def Profile_MostProb_Kmer(text:str,k: int,profile:list[dict[str,float]])-> str:
    # define a initial probability that is worse so that it definitely will update in the next step
    bestprob = -1
    num = len(text)- len(profile) +1
    best =""
    # traverse every possible start point of the k-mer
    for i in range(num):
        prob = 1.0
        motif = text[i:i+len(profile)]
        for j in range(k):
    #calculate the product of probability of each column
            prob *= profile[j][motif[j]]
 # output the motifs with the largest probability
        if prob > bestprob:
            best = motif
            bestprob = prob
    return best


# find out the most-probable motifs for multiple sequences
def Motif(dna:list[str], k: int, profile: list[dict[str,float]])->list[str]:
    best = []
    for i in range(len(dna)):
        motif = Profile_MostProb_Kmer(dna[i],k,profile)
        best.append(motif)
    return best


# sum the number of elements that are disagree with the consensus motifs
def score(motif:list[str]) -> int:
    M_score = 0
    # for every column
    for j in range(len(motif[0])):
        dictionary = {"A":0,"C":0,"G":0,"T":0}
    # for every row, count the frequency of each nucleotide
        for i in range(len(motif)):
            text = motif[i]
            dictionary[text[j]] += 1
 # the highest frequency is the element of the consensus motifs
        consensus = max(dictionary[i] for i in ["A","C","G","T"])
        different = sum(dictionary[i] for i in ["A","C","G","T"]) - consensus
        M_score += different
    return M_score